In [41]:
import re
import os
import string
import unicodedata
import contractions
import pandas as pd

In [42]:
TWITTER_RAW_DATA = os.path.join(os.getcwd(), "TwitterDataset.csv")
TWITTER_RAW_DATA

'd:\\Projects\\Machine Learning\\Few-Shot-GNN-LLM\\data-related\\TwitterDataset.csv'

In [ ]:
def text_regularization(text):
    try:
        text = str(text)
        # Convert to lowercase
        text = text.lower()

        # Expand contractions, can't => cannot
        text = contractions.fix(text)

        # Remove punctuations
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove special characters
        text = re.sub(r'[^a-z\s]', '', text)

        # Normalize accented characters "café" → "cafe"
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

        # Remove extra white spaces
        text = " ".join(text.split())
        return text
    except Exception as e:
        print('text_regularization', e)
        

In [ ]:
def preprocess_data(dataset_path = TWITTER_RAW_DATA):
    try:
        twitter_raw_data = pd.read_csv(dataset_path, encoding="ISO-8859-1")
        twitter_data = twitter_raw_data[twitter_raw_data["gender:confidence"] >= 0.5]
        columns_to_keep = ['gender', 'description']
        columns_to_drop = [col for col in twitter_data if col not in columns_to_keep]
        twitter_data = twitter_data.drop(columns=columns_to_drop)
        twitter_data = twitter_data.dropna()
        twitter_data = twitter_data[twitter_data['gender'].isin(['male', 'female'])]
        feature = twitter_data['description']
        feature = [text_regularization(each) for each in feature]
        label = twitter_data['gender']
        return (feature, label)
    except Exception as e:
        print('preprocess_data', e)

feature, label = preprocess_data()


['i sing my own rhythm', 'i am the author of novels filled with family drama and romance', 'louis whining and squealing and all', 'mobile guy ers shazam google kleiner perkins yahoo sprint pcs airtouch air force stanford gsb uva dad husband brother golfer', 'ricky wilson the best frontmankaiser chiefs the best band xxxx thank you kaiser chiefs for an incredible year of gigs and memories to cherish always xxxxxxx', 'you do not know me', 'the secret of getting ahead is getting started', 'pll fan crazy about mcd ramen is bae', 'renaissance art historian university of nottingham fuelled by haribo partial to coffee and with a soft spot for renaissance china national teaching fellow', 'senior xixiimmxiv', 'i am just here for tp bo burnham and disney world', 'jmkm', 'over enthusiastic f fan model collector music fan and a film fanatic also an aspergian', 'artisan specializing in paper mache printmaking and fibre art art teacher and cat devotee find my page on fb httpstcogbstgicv', 'he bled an